In [1]:
import os, glob, base64
import datetime
import pandas as pd
import concurrent.futures
import google.generativeai as genai
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from dotenv import find_dotenv, load_dotenv
import json
import yfinance as yf

c:\Users\Max\Desktop\projects\quanticon\pyquant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.options.mode.chained_assignment = None

In [3]:
today_str = datetime.datetime.today().strftime('%Y-%m-%d')

# Configuration
SHEET_ID = "15IfaN1fei9P6BXt0Nj7Rdj7SedDoN_Puzgyb6gUboVQ"
SHEET_NAME = "Sheet1"
DEFAULT_MODEL = "gemini-2.5-flash-preview-05-20"

# Load environment variables from .env file
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# Check if GEMINI_API_KEY is loaded
if not os.getenv("GEMINI_API_KEY"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [7]:
def get_trade_ideas():
    today_minus_day = datetime.datetime.today() - datetime.timedelta(1)
    today_minus_day_str = today_minus_day.strftime('%Y-%m-%d')
    print(today_minus_day)

    f_name1 = f'./outputs/{today_str}/backtest_ideas.csv'
    f_name2 = f'./outputs/{today_minus_day_str}/backtest_ideas.csv'
    if os.path.isfile(f_name1): 
        ideas = pd.read_csv(f_name1)
        # return ideas
    elif os.path.isfile(f_name2):
        ideas = pd.read_csv(f_name2)
        # return ideas
    else:
        try:
            scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
            print("setting creds...")
            creds = ServiceAccountCredentials.from_json_keyfile_name(
                os.getenv("GOOGLE_SHEET_API_KEY"), scope
            )
            print('authorizing...')
            client = gspread.authorize(creds)
            print("opening spreadsheet...")
            # sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
            spreadsheet = client.open_by_key(SHEET_ID)
            print("opening sheet...")

            sheet = spreadsheet.worksheet(SHEET_NAME)
            sheet_values = sheet.get_all_values()
            ideas_raw = pd.DataFrame(sheet_values[1:], columns=sheet_values[0])
            ideas_raw['date'] = pd.to_datetime(ideas_raw['date'])
            ideas = ideas_raw.loc[ideas_raw['date']<today_minus_day]
            ideas = ideas.loc[(ideas['date of outcome'].isna())|(ideas['date of outcome']=='')|(ideas['date of outcome']=='ongoing')]
            # print(ideas)
            ideas.to_csv(f'./outputs/{today_str}/backtest_ideas.csv', index=False)
        except Exception as e:
            print(f'Error getting sheet: {e}')
            ideas = pd.DataFrame()
    
    return ideas

def extract_last_60_days(ticker):
    ticker_data = yf.download(ticker, period='1y', interval='1d', group_by='ticker')
    df = ticker_data.tail(60)
    return df

def csv_to_base64(df:pd.DataFrame):
    csv_string = df.to_string()
    return csv_string

def call_idea_parse_agent(idea):
    # print('configuring gemini...')
    # api_key = os.getenv("GEMINI_API_KEY")
    # genai.configure(api_key=api_key)
    # print('gemini configured. init gemini...')
    llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    system_message = '''You are a helpful text parsing assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    user_message = f"""{idea}\n\nAnalyze this trade idea and parse the ticker (ticker only, no special characters allowed), the entry, """
    user_message += """the stop loss, and the target. Your output should simply be a json object like so: \n\n"""
    user_message += """{{"ticker":<parsed ticker>, "entry":<parsed entry>, "stop loss":<parsed stop loss>, "target":<parsed target>}} \n\n"""
    user_message += """There should be absolutely no other commentary, only the requested information."""
    # print('starting messages')
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", user_message)
    ])

    # print('created messages')
    chain = prompt | llm
    response = chain.invoke({"idea": idea})
    # print('invoked chain')
    return response.content

def call_analysis_agent(idea, date):

    idea_obj_raw = call_idea_parse_agent(idea)
    idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    json_idea = json.loads(idea_obj)

    price_history = extract_last_60_days(json_idea['ticker'])
    prices = price_history[json_idea['ticker']].reset_index()
    # print(prices)
    recent_prices = prices.loc[prices['Date']>date]
    recent_prices['entry'] = float(json_idea['entry'])
    recent_prices['sl'] = float(json_idea['stop loss'])
    recent_prices['tp'] = float(json_idea['target'])
    recent_prices['direction'] = 'bull' if json_idea['target'] > json_idea['entry'] else 'bear'
    
    # entries
    recent_prices['trade_entered'] = 0
    ef1 = recent_prices['direction'] == 'bull'
    ef2 = recent_prices['High'] >= recent_prices['entry']
    ef3 = recent_prices['direction'] == 'bear'
    ef4 = recent_prices['Low'] <= recent_prices['entry']
    recent_prices['trade_entered'] = recent_prices['trade_entered'].mask(((ef1)&(ef2))|((ef3)&(ef4)),1)
    
    # SLs
    recent_prices['sl_hit'] = 0
    sf1 = recent_prices['direction'] == 'bull'
    sf2 = recent_prices['Low'] <= recent_prices['sl']
    sf3 = recent_prices['direction'] == 'bear'
    sf4 = recent_prices['High'] >= recent_prices['sl']
    recent_prices['sl_hit'] = recent_prices['sl_hit'].mask(((sf1)&(sf2))|((sf3)&(sf4)),1)

    # TPs
    recent_prices['tp_hit'] = 0
    tf1 = recent_prices['direction'] == 'bull'
    tf2 = recent_prices['High'] >= recent_prices['tp']
    tf3 = recent_prices['direction'] == 'bear'
    tf4 = recent_prices['Low'] <= recent_prices['tp']
    recent_prices['tp_hit'] = recent_prices['tp_hit'].mask(((tf1)&(tf2))|((tf3)&(tf4)),1)

    analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})

    # price_history_str = csv_to_base64(analysis_gb)

    # llm = ChatGoogleGenerativeAI(model=DEFAULT_MODEL, temperature=0, api_key=GEMINI_API_KEY)

    # system_message = '''You are a helpful stock backtest analysis assistant. Your outputs are always in json, and you never provide any extra commentary beyond '''
    # system_message += '''what the user requests. It's critical that your ouput is always json, as it's meant to be consumed later by other APIs.'''
    # user_message = f"""<idea>{json_idea}</idea><price summary>{price_history_str}</price summary>\n\n"""
    # user_message = f"""Analyze this trade idea, along with the pricing summary, and tell me if 1 of the following occurred: - TP was hit\n- SL """
    # user_message += """was hit\n- the trade is still going\n- never entered trade\n\nAlso, please tell me the date that this occurred on."""
    # user_message += """There should be absolutely no other commentary, only the requested information."""
    # user_message += """Your output should be in the following json format: \n\n"""
    # user_message += """{{"outcome":<simple response>, "date of outcome": <YYYY-MM-DD for date outcome occurred>}} """
    # # print('starting messages')
    # prompt = ChatPromptTemplate.from_messages([
    #     ("system", system_message),
    #     ("user", user_message)
    # ])

    # # print('created messages')
    # chain = prompt | llm
    # response = chain.invoke({"price_history_str": price_history_str, "json_idea":json.dumps(json_idea)})
    # # print('invoked chain')
    # return response.content
    return analysis_gb, json_idea['ticker']

def process_raw_analysis(analysis_raw):
    try:
        trades2 = analysis_raw.reset_index()
        trades3 = trades2.loc[trades2['trade_entered']==1]
        trades3 = trades3.sort_values(['ticker', 'Date'])
        trades4 = trades3.loc[(trades3['sl_hit']>0)|(trades3['tp_hit']>0)]
        trades4['outcome'] = None
        trades4['outcome'] = trades4['outcome'].mask(trades4['tp_hit']==1,'TP Hit')
        trades4['outcome'] = trades4['outcome'].mask(trades4['sl_hit']==1,'SL Hit')
        trades5 = trades4.groupby(['ticker', 'outcome'])['Date'].min().reset_index()
        trades5 = trades5.sort_values(['ticker', 'Date'])
        # completed trades
        trades5 = trades5.drop_duplicates('ticker')
    except:
        trades5 = pd.DataFrame()
    try:
        # ongoing trades
        trades6 = trades3.loc[(trades3['sl_hit']==0)&(trades3['tp_hit']==0)&(~trades3['ticker'].isin(trades5['ticker']))]
    except:
        trades6 = pd.DataFrame()

    trades = pd.concat([trades5,trades6])
    return trades

def update_google_sheet(date_str, play_text):
    return

def process_ideas(df):
    # ai_output = ""  # Initialize ai_output
    # date_str = datetime.datetime.now().strftime("%Y-%m-%d")
    # failure_step = 'last 60'

    # try:
    #     df_last60 = extract_last_60_days(file_path)
        
    #     failure_step = 'csv conversion'
    #     base64_data = csv_to_base64(df_last60)
        
    #     failure_step = 'call agent'
    #     ai_output = call_agent(base64_data)
        
    #     failure_step = 'append to sheet'
    #     append_to_google_sheet(date_str, ai_output)
    #     print(f"Successfully processed {os.path.basename(file_path)}")
    #     return ai_output  # No error
    # except Exception as e:
    #     error_message = f"Error processing {os.path.basename(file_path)} on {date_str}: {e}\nAI Output: {ai_output}\n\n"
    #     print(error_message)
    #     print(f'step failed: {failure_step}')
    #     print('=====')
    #     return error_message
    return

In [8]:
ideas = get_trade_ideas()
ideas

2025-08-29 19:19:39.042207
setting creds...
authorizing...
opening spreadsheet...
opening sheet...


,date,play,date of outcome,outcome
121,2025-08-03,Ticker: $MLTX\nTrade Type: Swing\nBias: Bullis...,ongoing,ongoing
140,2025-08-16,Ticker: $TWST\nTrade Type: Scalp\nBias: Bearis...,ongoing,ongoing
141,2025-08-16,Ticker: $WHD\nTrade Type: Swing\nBias: Bearish...,ongoing,ongoing
142,2025-08-16,Ticker: UHAL\nTrade Type: Swing\nBias: Bearish...,ongoing,ongoing
143,2025-08-16,Ticker: TFIN\nTrade Type: Scalp\nBias: Bearish...,ongoing,ongoing
144,2025-08-16,Ticker: $TDW\nTrade Type: Swing\nBias: Bullish...,ongoing,ongoing
150,2025-08-16,Ticker: $KIDS\nTrade Type: Scalp\nBias: Bearis...,ongoing,ongoing
151,2025-08-16,Ticker: $ICHR\nTrade Type: Scalp\nBias: Bearis...,ongoing,ongoing
155,2025-08-16,Ticker: $FORM\nTrade Type: Scalp\nBias: Bearis...,ongoing,ongoing
157,2025-08-16,Ticker: $DAR\nTrade Type: Swing\nBias: Bearish...,ongoing,ongoing


In [9]:
import time

trades = pd.DataFrame()

for (i, row) in ideas.iterrows():
    # print(i)
    # print(row['play'])
    # idea_obj_raw = call_idea_parse_agent(row['play'])
    # idea_obj = idea_obj_raw.replace('```json','').replace('```','')
    # json_idea = json.loads(idea_obj)
    # # print(json_idea, json_idea['ticker'])
    # price_history = extract_last_60_days(json_idea['ticker'])
    # price_history_str = csv_to_base64(price_history)

    # trades.append([row['date'],json_idea,price_history_str])
    print(row['date'])
    print(row['play'])
    analysis_raw, ticker = call_analysis_agent(row['play'], row['date'])
    analysis_raw['ticker'] = ticker
    # analysis_gb = analysis_raw.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})
    
    print(analysis_raw)
    analysis_final = process_raw_analysis(analysis_raw)
    send_date = analysis_final.loc[analysis_final['ticker']==ticker, ['Date']]
    send_outcome = analysis_final.loc[analysis_final['ticker']==ticker, ['outcome']]
    new_row_data = [row['date'], row['play'], send_date, send_outcome]
    trades = pd.concat([trades, analysis_raw])
    # analysis_obj = analysis_raw.replace('```json','').replace('```','')
    # json_analysis = json.loads(analysis_obj)
    # print(json_analysis)
    # trades.append([row['date'],row['play'],json_analysis['date of outcome'],json_analysis['outcome']])
    time.sleep(2)

trades

2025-08-03 00:00:00
Ticker: $MLTX
Trade Type: Swing
Bias: Bullish
Entry: $52.92
Stop-Loss: $49.90
Target: $57.00
Option Contract: $55 Sep 19, 2025 Call
Rationale: MLTX shows a strong bullish reversal candle on increased volume, confirming a bounce from a healthy pullback within an established uptrend. All EMAs are aligned and trending upwards, supported by a bull_entry signal.


[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12036\3600393578.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price                             Date ticker
trade_entered sl_hit tp_hit                  
0             0      0      2025-08-07   MLTX
1             0      0      2025-08-04   MLTX
2025-08-16 00:00:00
Ticker: $TWST
Trade Type: Scalp
Bias: Bearish
Entry: $27.50
Stop-Loss: $29.00
Target: $25.00
Option Contract: $28 P 2025-08-22
Rationale: TWST is in a strong downtrend, recently experiencing a sharp breakdown. The current price action appears to be a dead cat bounce, with a fresh 'bear_entry' signal on the last day confirming renewed bearish momentum for a continuation lower.


[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12036\3600393578.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price                             Date ticker
trade_entered sl_hit tp_hit                  
0             0      0      2025-08-18   TWST
1             0      0      2025-08-19   TWST
2025-08-16 00:00:00
Ticker: $WHD
Trade Type: Swing
Bias: Bearish
Entry: $39.50
Stop-Loss: $40.50
Target: $37.00
Option Contract: $39.00 Puts, September 2025 Expiry
Rationale: WHD is in a strong downtrend, confirmed by bearish alignment of all EMAs and negative momentum deltas. The recent bounce from lows was rejected at prior resistance, and the latest price action shows a bearish continuation with a `bear_entry` signal, indicating further downside.


...

[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12036\3600393578.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price                             Date ticker
trade_entered sl_hit tp_hit                  
1             0      0      2025-08-25    ALB
                     1      2025-08-27    ALB
2025-08-22 00:00:00
Ticker: $ASO
Trade Type: Scalp
Bias: Bullish
Entry: $53.95
Stop-Loss: $51.40
Target: $57.00
Option Contract: $54 2025-08-29 Call
Rationale: ASO printed a strong bullish candle on 2025-08-22 with increased volume, breaking out of a short consolidation range after a pullback. This is confirmed by a `bull_entry` signal and positive `week_delta`, indicating a potential continuation of the broader uptrend.


[*********************100%***********************]  1 of 1 completed
C:\Users\Max\AppData\Local\Temp\ipykernel_12036\3600393578.py:115: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  analysis_gb = recent_prices.groupby(['trade_entered', 'sl_hit', 'tp_hit']).agg({'Date':min})


Price                             Date ticker
trade_entered sl_hit tp_hit                  
0             0      0      2025-08-29    ASO
1             0      0      2025-08-25    ASO
                     1      2025-08-28    ASO


Price                             Date ticker
trade_entered sl_hit tp_hit                  
0             0      0      2025-08-07   MLTX
1             0      0      2025-08-04   MLTX
0             0      0      2025-08-18   TWST
1             0      0      2025-08-19   TWST
0             1      0      2025-08-22    WHD
...                                ...    ...
1             0      0      2025-08-25    ALB
                     1      2025-08-27    ALB
0             0      0      2025-08-29    ASO
1             0      0      2025-08-25    ASO
                     1      2025-08-28    ASO

[64 rows x 2 columns]

In [10]:
trades2 = trades.reset_index()
trades3 = trades2.loc[trades2['trade_entered']==1]
trades3 = trades3.sort_values(['ticker', 'Date'])
trades3

Price,trade_entered,sl_hit,tp_hit,Date,ticker
59,1,0,0,2025-08-25,ALB
60,1,0,1,2025-08-27,ALB
62,1,0,0,2025-08-25,ASO
63,1,0,1,2025-08-28,ASO
25,1,0,0,2025-08-19,BWIN
58,1,0,0,2025-08-27,CROX
22,1,0,0,2025-08-18,DAR
56,1,0,0,2025-08-25,ERJ
19,1,0,0,2025-08-20,FORM
16,1,0,0,2025-08-20,ICHR


In [11]:
# completed trades
trades4 = trades3.loc[(trades3['sl_hit']>0)|(trades3['tp_hit']>0)]
trades4['outcome'] = None
trades4['outcome'] = trades4['outcome'].mask(trades4['tp_hit']==1,'TP Hit')
trades4['outcome'] = trades4['outcome'].mask(trades4['sl_hit']==1,'SL Hit')
trades5 = trades4.groupby(['ticker', 'outcome'])['Date'].min().reset_index()
trades5 = trades5.sort_values(['ticker', 'Date'])
trades5 = trades5.drop_duplicates('ticker')
trades5

,ticker,outcome,Date
0,ALB,TP Hit,2025-08-27
1,ASO,TP Hit,2025-08-28
2,IOT,TP Hit,2025-08-28
3,KTOS,TP Hit,2025-08-26
4,LTBR,TP Hit,2025-08-26
5,METC,TP Hit,2025-08-28
6,OUST,SL Hit,2025-08-25
7,RMBS,TP Hit,2025-08-28
8,SEI,TP Hit,2025-08-28
9,URBN,SL Hit,2025-08-28


In [12]:
# entered trades that are still ongoing
trades3.loc[(trades3['sl_hit']==0)&(trades3['tp_hit']==0)&(~trades3['ticker'].isin(trades5['ticker']))]

Price,trade_entered,sl_hit,tp_hit,Date,ticker
25,1,0,0,2025-08-19,BWIN
58,1,0,0,2025-08-27,CROX
22,1,0,0,2025-08-18,DAR
56,1,0,0,2025-08-25,ERJ
19,1,0,0,2025-08-20,FORM
16,1,0,0,2025-08-20,ICHR
13,1,0,0,2025-08-18,KIDS
1,1,0,0,2025-08-04,MLTX
42,1,0,0,2025-08-25,MTCH
11,1,0,0,2025-08-18,TDW
